# IRMA Scheduler Tutorial

Welcome to the Interactive Tutorial for the **IRMA (Installation Resource Management & Analytics) Scheduler**!

This notebook aims to guide new users/readers through the Mixed Integer Linear Programming (MILP) of the scheduler.

## Introduction
Imagine you are planning the schedule of installation activities for the deployment of an offshore system. 

📋 **Many Tasks**: Discrete work activities that need to be completed (e.g., anchor installation, cable laying)  
🚢 **Multiple Assets**: Resources (typically vessels) that can perform certain Tasks based on their capabilities (e.g., heavy-lift vessel, cable-laying vessel, support vessel)  
⚙️ **Task-Asset Assignments**: Multiple Assets can be assigned to each Task, where each assignment has an associated cost and time duration  
🌊 **Weather Windows**: Assets can only work in suitable sea conditions (e.g., wind speed limits, wave height limits)  
💰 **Time and Cost**: The schedule should aim to minimize the total cost of all tasks performed

How can the scheduler figure out what assets to assign to what tasks and when to perform each task to minimize time and/or cost?

### Mixed Integer Linear Programming (MILP)

A type of optimization problem that uses a mixture of integer, binary, and continuous variables subject to linear constraints to minimize an objective. 

As an example, let's say you own a truck delivery company with 3 trucks and you need to decide which trucks to send out for delivery, where each truck ($x_i$) has a cost of delivery and a time duration. The goal is to minimize cost of delivery, under a constraint that the total delivery time needs to be at least 12 hours. The only decisions are to either send out the truck for delivery (1) or not (0).

Minimize the cost function
$$ 500x_1 + 400x_2 + 300x_3 $$

subject to specific time constraints (a little counterintuitive that we need a lower bound on time, but it'll help with the whole tutorial)

$$ 7x_1 + 6x_2 + 4x_3 \geq 12 $$

A MILP solver will realize that it needs at least two trucks for delivery to satisfy the time constraint and it will also figure out that Truck 2 ($x_2$) and Truck 3 ($x_3$) will not satisfy the constraint and neither will Truck 1 ($x_1$) and Truck 3 ($x_3$). That leaves the options of Truck 1 and Truck 2, or all three trucks. It will choose only Truck 1 and Truck 2 since that minimizes cost. (This also assumes that each truck can only be used once).

This tutorial only considers binary variables (for now),

$$x \in \{0,1\}$$

to determine what decisions to make to minimize the objective of the scheduler.

## Setup

First, let's import the necessary libraries and set up our environment.

In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
import os

# Add the FAModel path for imports
sys.path.append(os.path.join(os.getcwd(), 'famodel'))

# Import the IRMA scheduler
from famodel.irma.scheduler import Scheduler

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = [12, 8]

print("✅ Libraries imported successfully!")
print("📊 Ready to explore the IRMA Scheduler!")

## Simple Case: Two Tasks, Two Assets

Let's start with the most basic scenario to understand the fundamentals:

### Tasks

Let's say that the installation of an offshore system requires two tasks: installing a mooring line, and installing an anchor, where each task has certain requirements that are needed to complete the task.

In [ ]:
tasks = [
    {
        'name': "install_mooring",
        'requirements': ['mooring_reel', 'positioning']
    },
    {
        'name': "install_anchor",
        'requirements': ['anchor_handling','positioning']
    }
]

# Display task information
task_df = pd.DataFrame(tasks)
print(task_df)

### Assets

And that there are two vessels (assets) that could potentially be used to perform these installations, each with their own set of capabilities, daily cost, and an integer value to represent what weather conditions it can operate in. For example, the Multi-Purpose Supply Vessel (MPSV) cannot operate in wave heights greater than 2 m, but the Anchor Handling Tug Supply Vessel (AHTS) can, but no greater than 4 m.

In [ ]:
# Define our installation vessel
assets = [
    {
        'name': 'AHTS', 
        'capabilities': ['anchor_handling', 'mooring_reel', 'positioning'],
        'daily_cost': 50000,
        'max_weather': 2
    },
    {
        'name': 'MPSV', 
        'capabilities': ['mooring_reel', 'positioning'],
        'daily_cost': 25000,
        'max_weather': 1
    }
]

print("🚢 Asset Definition:")
asset_df = pd.DataFrame(assets)
print(asset_df)

### The Task-Asset Matrix

Through a process that is still yet to be determined (TODO...Stein has something started), we can generate a **Task-Asset Matrix** that defines the cost and duration to perform each task by each set of assets.

Each row of the task-asset matrix represents a different task and each column of the task-asset matrix represents a combination of assets.

Entries with values of -1 represent task-asset pairs that are not feasible. Something like installing an anchor with a kayak.

In [ ]:
#                 |  AHTS    |  MPSV    | AHTS+MPSV |
# Install Mooring | (c, d)   | (c, d)   |   (c, d)  |
# Install Anchor  | (c, d)   | (c, d)   |   (c, d)  |
task_asset_matrix = np.array([
                  [(2000, 2), (1000, 3), (2500, 3)],
                  [(1500, 3), (-1,  -1), (4000, 2)]
])

print("🔗 Task-Asset Compatibility:")
print(task_asset_matrix)

### Asset Groups

Different combinations of assets can be used for each task, and each produce a different cost and duration to perform the task based on the capabilities of the assets and the requirements of the task.

The matrix generation process will filter out asset combinations that do not make sense (i.e., overlapping capabilities, maximum number of assets involved, extremely high costs, etc.)

In [ ]:
asset_groups = [
    {
        'assets': ['AHTS'], 
    },
    {
        'assets': ['MPSV'], 
    },
    {
        'assets': ['AHTS','MPSV'], 
    },
]

print("🚢 Asset Groups")
asset_group_df = pd.DataFrame(asset_groups)
print(asset_group_df)

### Time Periods & Weather

We can also define the planning horizon (timeline) as a set of time periods with given weather conditions. Time periods could be any duration of time (e.g., hours, days, weeks, etc.).

Good weather is normally designated by a 1, OK weather is designated by a 2, and bad weather is designated by a 3

In [ ]:
weather = [1, 1, 1, 1, 1]   # Start by defining 5 time periods, each with good weather

print("📅 Planning Horizon:")
weather_df = pd.DataFrame({
    'Weather_Condition': weather,
    'Description': ['Good weather'] * len(weather)
})
print(weather_df)

### Running the Simple Scheduler

Now let's create and run our first scheduler instance, which simply sets up many variables within the Scheduler class.

In [ ]:
# Create the scheduler for our simple scenario
print("🔧 Creating scheduler for simple scenario...")

scheduler = Scheduler(
    tasks=tasks,
    assets=assets,
    task_asset_matrix=task_asset_matrix,
    weather=weather,
    asset_groups=asset_groups,
    wordy=0  # Enable some debug output
)

print("✅ Scheduler created successfully!")

## Understanding the Mathematical Constraints

Before we dive into the results of the scheduler, let's understand how the IRMA scheduler actually works under the hood

### Decision Variables:

The scheduler initialization uses three types of binary decision variables in the MILP optimization (each can be 0 or 1):

**📋 Task-Asset Assignment Variables** `Xta[t,a]`: determines whether task $t$ is assigned to asset group $a$
- `Xta[0,1] = 1` means "Task 0 is assigned to Asset 1"
- `Xta[0,1] = 0` means "Task 0 is NOT assigned to Asset 1"

**⏰ Task-Period Activity Variables** `Xtp[t,p]`: determines if task $t$ is active in period $p$
- `Xtp[0,3] = 1` means "Task 0 is active during Period 3"  
- `Xtp[0,3] = 0` means "Task 0 is NOT active during Period 3"

**🚀 Task Start Time Variables** `Xts[t,s]`: determines if task $t$ starts at period $s$
- `Xts[0,2] = 1` means "Task 0 starts at Period 2"
- `Xts[0,2] = 0` means "Task 0 does NOT start at Period 2"

The only 'decisions' are whether certain asset groups are assigned to certain tasks $(X_{t,a})$, and when the task starts $(X_{t,vas}r)i$a.bles  The $X_{t,p}$ are also included to help organize the constraints in determining what periods each task occupy, based on the duration of the task-asset combination defined in the task-asset matrix.

The full decision variable vector $x$ then follows the form of 

$$ x = [X_{t,a}, X_{t,p}, X_{t,s}] $$

where the length depends on the number of tasks $T$, the number of asset groups $A$, and the number of periods $P$


### Constraints

Each constraint ensures the solution makes logical sense. In an MILP optimization, constraints are set to be linear and follow the form of 

$$
 A_{ub} \text{ } x \text{ } \leq b_{ub} \\
 A_{eq} \text{ } x \text{ } = b_{eq}    \\
 A_{lb} \text{ } x \text{ } \geq b_{lb} \\
$$

where $A$ and $b$ represent large vectors that when multipled by the binary decision variables of the $x$ vector, need to satisfy the constraint according to $b$.

In the example at the top of this tutortial, we would structure the constraint as the following:

$$
\begin{bmatrix}
7 & 6 & 4
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 \\ x_3
\end{bmatrix}
\ge 12
$$

This format is used to define many other constraints necessary for a logical sotion for the scheduler. Each constraint is explained below. But first, we run the `set_up_optimizer()` function to create all the constraints that we can then analyze through this tutorial.

In [ ]:
scheduler.set_up_optimizer()

#### Constraint 1: Task-Asset Validity 🔒

**English**: An asset group can only be assigned to a task if the asset group can perform the task.

**Impact**: Prevents impossible assignments (Xta variables that correspond to invalid entries in the task-asset matrix from being turned on) that would break the physics of the problem.

**Math**: 
$$
X_{t,a} = 0 \quad \text{for all } (t,a) \text{ where } c_{t,a} < 0 \text{ or } d_{t,a} < 0
$$

**Implementation** for the current example:

In [ ]:
# Constraint 1: Task-Asset Validity Matrix Construction
print("🔒 Constraint 1: Task-Asset Validity")
print()

if hasattr(scheduler, 'A_eq_1'):
    print(f"A_eq_1 matrix equations (shape: {scheduler.A_eq_1.shape}):")
    for i, (row, b_val) in enumerate(zip(scheduler.A_eq_1, scheduler.b_eq_1)):
        row_str = '[' + ' '.join([str(val) for val in row]) + ']'
        print(f"{row_str} x = {b_val}")
else:
    print("No invalid task-asset pairs found - no constraints needed")

In this example, we are constraining the system so that

$$
X_{t,a}[1,1] = 0
$$

since that is the only entry of the task-asset matrix that is infeasible

#### Constraint 3: Exactly One Asset Per Task ⚖️

**English**: Each task must be assigned to exactly one asset group

**Impact**: Prevents a task from being unassigned (would never complete) or over-assigned (physically impossible).

**Math**: 
$$
\sum_{a=0}^{A-1} X_{t,a} = 1 \quad \forall t \in \{0, 1, \ldots, T-1\}
$$

**Implementation** for the current example:

In [ ]:
# Constraint 3: Exactly One Asset Per Task Matrix Construction
print("⚖️ Constraint 3: Exactly One Asset Per Task")
print()

print(f"A_eq_3 matrix equations (shape: {scheduler.A_eq_3.shape}):")
for i, (row, b_val) in enumerate(zip(scheduler.A_eq_3, scheduler.b_eq_3)):
    row_str = '[' + ' '.join([str(val) for val in row]) + ']'
    print(f"{row_str} x = {b_val}")

In this example, we are constraining the system so that

$$ X_{t,a}[0,0] + X_{t,a}[0,1] + X_{t,a}[0,2] = 1 $$ and $$ X_{t,a}[1,0] + X_{t,a}[1,1] + X_{t,a}[1,2] = 1 $$

which doesn't allow more than 1 asset group assignment per task


#### Constraint 15: Each Task Must Have A Start Time 🚀

**English**: Every task must start exactly once

**Impact**: Tasks must start to be completed, and they can only start once.

**Math**: 
$$
\sum_{s=0}^{S-1} X_{t,s} = 1 \quad \forall t \in \{0, 1, \ldots, T-1\}
$$

**Implementation** for the current example:

In [ ]:
# Constraint 15: Each Task Must Have A Start Time Matrix Construction
print("🚀 Constraint 15: Each Task Must Have A Start Time")
print()

print(f"A_eq_15 matrix equations (shape: {scheduler.A_eq_15.shape}):")
for i, (row, b_val) in enumerate(zip(scheduler.A_eq_15, scheduler.b_eq_15)):
    row_str = '[' + ' '.join([str(val) for val in row]) + ']'
    print(f"{row_str} x = {b_val}")

In this example, we are constraining the system so that

$$ X_{t,s}[0,0] + X_{t,s}[0,1] + X_{t,s}[0,2] + X_{t,s}[0,3] + X_{t,s}[0,4] = 1 $$ and $$ X_{t,s}[1,0] + X_{t,s}[1,1] + X_{t,s}[1,2] + X_{t,s}[1,3] + X_{t,s}[1,4] = 1 $$

which doesn't allow more than 1 start time per task

#### Constraint 10: Task Duration Must Not Exceed Planning Horizon 📅

**English**: A task cannot start at a time where its duration would extend beyond the available planning periods.

**Impact**: Prevents scheduling tasks that would run past the end of the planning window, ensuring all work completes within the defined timeframe.

**Math**: 
$$
X_{t,a}[t,a] + X_{t,s}[t,s] \leq 1 \quad \forall t, a, s \quad \text{ where } \quad d_{t,a} > 0 \text{ and } s + d_{t,a} > P
$$

**Implementation** for the current example:

In [ ]:
# Constraint 10: Task Duration Must Not Exceed Planning Horizon Matrix Construction
print("📅 Constraint 10: Task Duration Must Not Exceed Planning Horizon")
print()

if hasattr(scheduler, 'A_ub_10'):
    print(f"A_ub_10 matrix equations (shape: {scheduler.A_ub_10.shape}):")
    for i, (row, b_val) in enumerate(zip(scheduler.A_ub_10, scheduler.b_ub_10)):
        row_str = '[' + ' '.join([str(val) for val in row]) + ']'
        print(f"{row_str} x ≤ {b_val}")
else:
    print("No duration violations found - all tasks can complete within planning horizon")

In this example, we are constraining the system so that 

$ X_{t,a}[0,0] + X_{t,s}[0,4] \leq 1 \quad X_{t,a}[0,1] + X_{t,s}[0,3] \leq 1 \quad X_{t,a}[0,1] + X_{t,s}[0,4] \leq 1 $

which says that if the first asset group is assigned to the first task, then the first task cannot start in the last period (because its duration is 2 periods). Similarly, if the second asset group is assigned to the first task, then the first task cannot start in the fourth or fifth periods (because its duration is 3 periods).

#### Constraint 14a: Task Must Be Active When It Starts ⏰

**English**: If a task starts in a specific period, it must also be active in that same period.

**Impact**: Links start time decisions to activity periods - ensures that when a task starts, it's immediately active.

**Math**: 
$$
X_{t,s}[t,s] \leq X_{t,p}[t,p] \quad \forall t, s \quad \text{ where } \quad s = p \text{ and } s < P
$$

**Implementation** for the current example:

In [ ]:
# Constraint 14a: Task Must Be Active When It Starts Matrix Construction
print("⏰ Constraint 14a: Task Must Be Active When It Starts")
print()

if hasattr(scheduler, 'A_ub_14a'):
    print(f"A_ub_14a matrix equations (shape: {scheduler.A_ub_14a.shape}):")
    for i, (row, b_val) in enumerate(zip(scheduler.A_ub_14a, scheduler.b_ub_14a)):
        row_str = '[' + ' '.join([str(val) for val in row]) + ']'
        print(f"{row_str} x ≤ {b_val}")
else:
    print("No start-time to period mapping constraints needed")

In this example, we are constraining the system so that 

$ -X_{t,p}[0,0] + X_{t,s}[0,0] \leq 0 \quad -X_{t,p}[0,1] + X_{t,s}[0,1] \leq 0 \quad -X_{t,p}[0,2] + X_{t,s}[0,2] \leq 0 $

which says that if the first task starts in the first time period, then the Xtp variable that corresponds to that start period must also be turned on.

#### Constraint 14b: Task Activity Must Match Duration ⏱️

**English**: If a task is assigned to an asset and starts at a specific time, it must be active for exactly the duration required by that task-asset combination.

**Impact**: Ensures tasks run for their complete required duration based on the chosen asset assignment and start time.

**Math**: 
$$
X_{t,a}[t,a] + X_{t,s}[t,s] - X_{t,p}[t,p] \leq 1 \quad \forall t, a, s, p \quad \text{ where } \quad d_{t,a} > 0 \text{ and } s \leq p < s + d_{t,a}
$$

**Implementation** for the current example:

In [ ]:
# Constraint 14b: Task Activity Must Match Duration Matrix Construction
print("⏱️ Constraint 14b: Task Activity Must Match Duration")
print()

if hasattr(scheduler, 'A_ub_14b'):
    print(f"A_ub_14b matrix equations (shape: {scheduler.A_ub_14b.shape}):")
    for i, (row, b_val) in enumerate(zip(scheduler.A_ub_14b, scheduler.b_ub_14b)):
        row_str = '[' + ' '.join([str(val) for val in row]) + ']'
        print(f"{row_str} x ≤ {b_val}")
else:
    print("No duration enforcement constraints needed")

In this example, we are constraining the system so that 

$ X_{t,a}[0,0] - X_{t,p}[0,0] + X_{t,s}[0,0] \leq 1 \quad X_{t,a}[0,0] - X_{t,p}[0,1] + X_{t,s}[0,0] \leq 1 \quad X_{t,a}[0,0] - X_{t,p}[0,1] + X_{t,s}[0,1] \leq 1$

which ensures that for each case when a task-asset group starts in a certain time period, the Xtp variables that would align with the start time period and the duration of the task are turned on. The list of constraints follows a pattern where it loops through all task-asset group options, and then loops through all start time options, and uses a -1 coefficient on the Xtp variables that equate to the duration of the task. Constraint 14a only does the first Xtp variable corresponding to the start time. Constraint 14b ensures the Xtp variables that align with the duration of the task are also turned on.

#### Constraint 16: Each Task Active For Exactly Its Duration ⚖️

**English**: The total number of periods a task is active must exactly equal the duration required by its assigned asset group.

**Impact**: Prevents tasks from being active for longer or shorter than required, working with Constraint 14b to ensure precise duration matching.

**Math**: 
$$
\sum_{p=0}^{P-1} X_{t,p} = \sum_{a=0}^{A-1} X_{t,a} \cdot d_{t,a} \quad \forall t
$$

**Note**: Constraint 14b ensures tasks are active during their assigned periods, but doesn't necessarily prevent periods from being active outside of the ones that it specifies in this constraint. For example, if Task 0 uses Asset 1 with duration=3 and starts in Period 2, Constraint 14b ensures Periods 2, 3, and 4 are turned on, but doesn't prevent Task 0 from being active in Periods 0, 1, or 5. Constraint 16 ensures the sum of Xtp variables equals the duration exactly. This method does not seem the cleanest right now, but no other methods were found that were cleaner.

**Implementation** for the current example:

In [ ]:
# Constraint 16: Each Task Active For Exactly Its Duration Matrix Construction
print("⚖️ Constraint 16: Each Task Active For Exactly Its Duration")
print()

if hasattr(scheduler, 'A_eq_16'):
    print(f"A_eq_16 matrix equations (shape: {scheduler.A_eq_16.shape}):")
    for i, (row, b_val) in enumerate(zip(scheduler.A_eq_16, scheduler.b_eq_16)):
        row_str = '[' + ' '.join([str(val) for val in row]) + ']'
        print(f"{row_str} x = {b_val}")
else:
    print("No duration matching constraints needed")

In this example, we are constraining the system so that 

$$ -2X_{t,a}[0,0] - 3X_{t,a}[0,1] -3X_{t,a}[0,2] + X_{t,p}[0,0] + X_{t,p}[0,1] + X_{t,p}[0,2] + X_{t,p}[0,3] + X_{t,p}[0,4] = 0 $$
$$ -3X_{t,a}[1,0] - 3X_{t,a}[1,2] + X_{t,p}[1,0] + X_{t,p}[1,1] + X_{t,p}[1,2] + X_{t,p}[1,3] + X_{t,p}[1,4] = 0 $$

which ensures that no matter which Xta pair is selected, per task, that sum of the Xtp variables must equal the corresponding coefficient. If the first asset group is assigned to the first task, then there needs to be only two Xtp[0,p] variables turned on. Constraint 3 ensures that multiple Xta variables per task are not selected. This constraint is used because while Constraint 14b ensures the proper Xtp variables are turned on based on the task's duration, it has no control over the other Xtp variables outside of the start time period plus duration. This constraint provides the upper bound on those Xtp variables.

#### Constraint 4: Asset Conflict Prevention 🚫

**English**: Individual assets cannot be used by multiple tasks simultaneously, even when those assets are part of different asset groups.

**Impact**: Prevents physical resource conflicts where the same vessel would need to be in two places at once, ensuring realistic scheduling.

**Math**: 
$$
X_{t,a}[t_1,a_1] + X_{t,a}[t_2,a_2] + X_{t,p}[t_1,p] + X_{t,p}[t_2,p] \leq 3 \quad \forall t_1, t_2, a_1, a_2, p \text{ where individual assets overlap in asset groups}
$$

**Implementation** for the current example:

In [ ]:
# Constraint 4: Asset Conflict Prevention Matrix Construction
print("🚫 Constraint 4: Asset Conflict Prevention")
print()

if hasattr(scheduler, 'A_ub_4'):
    print(f"A_ub_4 matrix equations (shape: {scheduler.A_ub_4.shape}):")
    for i, (row, b_val) in enumerate(zip(scheduler.A_ub_4, scheduler.b_ub_4)):
        row_str = '[' + ' '.join([str(val) for val in row]) + ']'
        print(f"{row_str} x ≤ {b_val}")
else:
    print("No individual asset conflicts found - no constraints needed")

In this example, we are constraining the system so that 

$ X_{t,a}[0,0] + X_{t,a}[1,0] + X_{t,p}[0,0] + X_{t,p}[1,0] \leq 3 $

which ensures that if different tasks use the same asset (that are included in different asset groups), then that asset can only be used for one time period. In this case, if both tasks use the first asset group (which has the same assets between each other), then only one task can have their Xtp variables turned on because assets can't be doing two things at once. We also have other constraints like

$ X_{t,a}[0,0] + X_{t,a}[1,2] + X_{t,p}[0,0] + X_{t,p}[1,0] \leq 3 $

which is used because the third asset group has at least one of the same assets that the first asset group has, and so the same rules apply. The implementation checks for similar assets in different asset groups and creates the constraints based on those overlaps. The bound value of 3 in this example is a function of the number of tasks (1 + T).

### Constraint Summary 🔗

1. **Constraint 1** ensures only valid assignments are possible
2. **Constraint 3** ensures every task gets exactly one asset
3. **Constraint 15** ensures every task starts exactly once  
4. **Constraint 10** ensures the task duration does not exceed planning horizon
5. **Constraint 14a** links start time to activity period
6. **Constraint 14b** links start times to activity periods with correct duration
7. **Constraint 16** equates total activity periods to correct duration
8. **Constraint 4** prevents resource conflicts between asset groups

### Objectives and other MILP Inputs

Beyond the constraint matrices, the MILP optimizer requires three additional key inputs: the objective values vector, variable bounds, and integrality specifications.

#### 1. Objective Values Vector

The `values` vector defines the coefficients for the objective function that the optimizer seeks to minimize. In IRMA's scheduler, this represents the cost or penalty associated with each decision variable.

Each element corresponds to a decision variable and represents the "cost" of setting that variable to 1.

$$\text{minimize } \mathbf{c}^T \mathbf{x} = \sum_{i} c_i x_i$$

where $c_i$ is the cost coefficient and $x_i$ is the binary decision variable.

The costs of each task-asset combination are included as entries to the values vector for each coresponding Xta variable.

Penalties are also included for each Xts variable to incentivize earlier Xts variables to be selected rather than later.

In [ ]:
print(scheduler.values)

### 2. Bounds

The `bounds` parameter defines the lower and upper limits for each decision variable. In our case, all decision variables are binary (0 or 1), so bounds are set as:
```python
bounds = optimize.Bounds(0, 1)  # 0 ≤ x_i ≤ 1
```

**Mathematical Form**: For each variable $x_i$:
$$0 \leq x_i \leq 1$$


### 3. Integrality

The `integrality` parameter specifies which variables must take integer values. It forces variables to be integers rather than continuous. In our case, we are only working with binary integers so integrality is set to 1 for all variables:
```python
integrality = np.ones(num_variables, dtype=int)  # All variables are integers
```

**Mathematical Form**: Each variable $x_i$ must satisfy:
$$x_i \in \{0, 1\}$$

Combined with bounds [0,1], this creates binary decision variables that are either "not selected" (0) or "selected" (1).


## Running the Code

When you run `scheduler.optimize()`, the optimization engine:

1. **Builds** the constraint matrices (A_ub, A_eq, b_ub, b_eq) and MILP inputs
2. **Solves** the MILP problem using scipy.optimize.milp
3. **Returns** optimal values for all decision variables
4. **Decodes** the solution into human-readable schedules

An example of how the optimizer is called:

```python
from scipy.optimize import milp

res = milp(
    c=values,                # Objective function coefficients
    constraints=constraints, # List of LinearConstraint objects
    integrality=integrality, # Integer specification for each variable
    bounds=bounds           # Variable bounds (0 to 1 for binary)
)
```

This formulates and solves the complete Mixed Integer Linear Programming problem:
- **Minimize**: $\mathbf{c}^T \mathbf{x}$ (total cost)
- **Subject to**: All constraint equations
- **Where**: $x_i \in \{0, 1\}$ for all $i$ (binary decisions)

Let's see this in action with our simple example!

In [ ]:
# Solve the optimization problem
print("🚀 Solving the optimization problem...\n")

result = scheduler.optimize()

The results of the optimization provide a schedule for installation that minimizes cost and sastisfies all constraints!

- The optimization follows our penalty of starting tasks as soon as possible
- It decides to schedule the "Install Mooring" task in the first 3 periods using the MPSV asset
- It decides to schedule the "Install Anchor" task also in the first 3 periods but using a separate vessel, the AHTS asset, which is allowed.
- This combination of task-asset group assignments minimized cost and kept all of our logical constraints honored.

Now let's adjust the weather to see how that impacts the schedule and the limits of each asset group

### Constraint 17: Weather Restrictions 🌊

**English**: Asset groups cannot be assigned to tasks during periods when weather conditions exceed their operational limits.

**Impact**: Prevents scheduling tasks during unsuitable weather conditions, ensuring safety and operational limits are respected.

**Math**:
$$
X_{t,a}[t,a] + X_{t,p}[t,p] \leq 1 \quad \forall t, a, p \quad \text{ where } \quad w_p > \text{max\_weather}_a
$$

where:
- $w_p$ = weather severity in period $p$
- $\text{max\_weather}_a$ = minimum weather capability across all individual assets in asset group $a$

**Implementation** for our current example (WHILE UPDATING OUR WEATHER CONDITIONS):

In [ ]:
#scheduler.weather = [1, 2, 3, 3, 1]
scheduler.weather = [2, 3, 1, 1, 1]
scheduler.set_up_optimizer()

# Constraint 17: Weather Restrictions Matrix Construction
print("🌊 Constraint 17: Weather Restrictions")
print()

if hasattr(scheduler, 'A_ub_17'):
    print(f"A_ub_17 matrix equations (shape: {scheduler.A_ub_17.shape}):")
    for i, (row, b_val) in enumerate(zip(scheduler.A_ub_17, scheduler.b_ub_17)):
        row_str = '[' + ' '.join([str(val) for val in row]) + ']'
        print(f"{row_str} x ≤ {b_val}")
else:
    print("No weather restrictions needed - all asset groups can work in all conditions")

In this example, we have adjusted the weather conditions of the scenario (1, 2, 2, 3, 1), which adds additional constraints: 

$ X_{t,a}[0,0] + X_{t,p}[0,1] \leq 1 $

which does not allow the first task-asset group combination to be active in the second period, since that period has weather conditions that exceed the allowances of the minimum asset in that asset group. Similarly, there are other constraints like

$ X_{t,a}[0,2] + X_{t,p}[0,0] \leq 1 \quad X_{t,a}[0,2] + X_{t,p}[0,1] \leq 1 $

which does not allow the second asset group combined with the first task to be active in the first or second periods, since those periods have weather conditions that exceed the minimum allowance of the asset in that asset group.

Now, let's rerun the scheduler and see what impact it has.

In [ ]:
# Solve the optimization problem
print("🚀 Solving the optimization problem with adjusted weather conditions\n")

result = scheduler.optimize()

The results of this new optimization show how the weather impacts the schedule!

- Bad weather in period 1 does not allow any task-asset combination from being scheduled in that period
- This means that the tasks have to happen in periods 2, 3, and 4

Now let's adjust the weather back to the original calm status and add in dependency constraints

### Constraint 2: Task Dependencies 🔗

**English**: Tasks must be completed in a specific order based on their dependencies. For example, a task cannot start until all its prerequisite tasks have been completed.

**Impact**: Ensures logical sequencing of installation activities - for example, anchors must be installed before mooring lines can be connected to them.

**Math**: For finish_start dependency (most common type):
$$
X_{t,s}[t,s] \leq \sum_{s_d=0}^{s-d_{min}} X_{t,s}[d,s_d] \quad \forall t, s \quad \text{ where task } t \text{ depends on task } d
$$

where:
- $d_{min}$ = minimum duration of dependency task $d$ across all possible assets
- Task $t$ can only start at time $s$ if task $d$ started early enough to finish before time $s$

**Implementation** for our example, which requires re-initializing the scheduler:

In [ ]:
# Let's create a new scheduler instance with task dependencies
print("🔗 Constraint 2: Task Dependencies")
print()

# Define task dependencies: mooring installation depends on anchor installation
task_dependencies = {
    'install_mooring': ['install_anchor']  # Mooring installation depends on anchor installation
}

dependency_types = {
    'install_anchor->install_mooring': 'finish_start'  # Anchor must finish before mooring starts
}

print("📋 Task Dependencies:")
print(f"  install_mooring depends on: {task_dependencies['install_mooring']}")
print(f"  Dependency type: {dependency_types['install_anchor->install_mooring']}")
print()

# Create scheduler with dependencies
scheduler_with_deps = Scheduler(
    tasks=tasks,
    assets=assets,
    task_asset_matrix=task_asset_matrix,
    weather=[1, 1, 1, 1, 1],  # Reset to good weather
    asset_groups=asset_groups,
    task_dependencies=task_dependencies,
    dependency_types=dependency_types,
    wordy=0
)

scheduler_with_deps.set_up_optimizer()

if hasattr(scheduler_with_deps, 'A_ub_2'):
    print(f"\nA_ub_2 matrix equations (shape: {scheduler_with_deps.A_ub_2.shape}):")
    for i, (row, b_val) in enumerate(zip(scheduler_with_deps.A_ub_2, scheduler_with_deps.b_ub_2)):
        row_str = '[' + ' '.join([str(val) for val in row]) + ']'
        print(f"{row_str} x ≤ {b_val}")
        
    print(f"\nConstraint 2 ensures that the mooring installation task cannot start")
    print(f"until the anchor installation task has been completed, maintaining logical")
    print(f"installation sequencing.")
else:
    print("No task dependencies defined - no constraints needed")

In this example, we have updated the dependencies to ensure that Task 0 (Install Mooring) is only done after Task 1 (Install Anchor) is completed, which adds many additional constraints, like:

$ X_{t,a}[1,0] + X_{t,s}[0,0] + X_{t,s}[1,0] \leq 2 \quad X_{t,a}[1,0] + X_{t,s}[0,1] + X_{t,s}[1,0] \leq 2 \quad X_{t,a}[1,0] + X_{t,s}[0,2] + X_{t,s}[1,0] \leq 2$

which ensures that if the second task is active with the first asset group, then the first task cannot start in periods 0, 1, or 2 if the second task starts in period 0. (The names of first and second are confusing here, but it's written correctly).

This same logic applies for all other start times for each feasible task-asset combination for the dependent task, blocking it from occupying time periods that would violate the constraint.

In [ ]:
scheduler_with_deps.optimize()

### Constraint 2+: Task Dependencies with Offsets

We now showcase how we can add offsets to the dependencies.

As a basic example, how would the scheduling work if Task 0 (Install Mooring) were to start one period after Task 1 (Install Anchor) starts?

Let's run another example:

In [ ]:
# Define task dependencies: mooring installation depends on anchor installation
task_dependencies = {
    'install_mooring': ['install_anchor']  # Mooring installation depends on anchor installation
}

dependency_types = {
    'install_anchor->install_mooring': 'start_start'  # Anchor must finish before mooring starts
}

offsets = {
    'install_anchor->install_mooring': 1    # Mooring installation to start 1 period after Anchor installation
}

# Create scheduler with dependencies
scheduler_with_offset_dep = Scheduler(
    tasks=tasks,
    assets=assets,
    task_asset_matrix=task_asset_matrix,
    task_dependencies=task_dependencies,
    dependency_types=dependency_types,
    offsets=offsets,
    weather=[1, 1, 1, 1, 1],  # Reset to good weather
    asset_groups=asset_groups,
    wordy=0
)

scheduler_with_offset_dep.set_up_optimizer()

if hasattr(scheduler_with_deps, 'A_ub_2'):
    print(f"\nA_ub_2 matrix equations (shape: {scheduler_with_deps.A_ub_2.shape}):")
    for i, (row, b_val) in enumerate(zip(scheduler_with_deps.A_ub_2, scheduler_with_deps.b_ub_2)):
        row_str = '[' + ' '.join([str(val) for val in row]) + ']'
        print(f"{row_str} x ≤ {b_val}")
        
    print(f"\nConstraint 2 ensures that the mooring installation task cannot start")
    print(f"until the anchor installation task has been completed, maintaining logical")
    print(f"installation sequencing.")
else:
    print("No task dependencies defined - no constraints needed")

scheduler_with_offset_dep.optimize()

## A More Complicated Case

Provide an example of a more involved schedule problem with many more assets and tasks...